In [1]:
import qlib
from qlib.utils import init_instance_by_config
from qlib.data.dataset.loader import QlibDataLoader
from qlib.data.dataset.handler import DataHandlerLP
from qlib.config import REG_CN

In [2]:
provider_uri = "E:/qilb研究/.qlib_data/cn_data"
qlib.init(provider_uri=provider_uri, region=REG_CN, expression_cache=None, dataset_cache=None)

[15996:MainThread](2022-01-22 21:15:08,515) INFO - qlib.Initialization - [config.py:393] - default_conf: client.
[15996:MainThread](2022-01-22 21:15:10,462) INFO - qlib.Initialization - [__init__.py:57] - qlib successfully initialized based on client settings.
[15996:MainThread](2022-01-22 21:15:10,462) INFO - qlib.Initialization - [__init__.py:59] - data_path={'__DEFAULT_FREQ': WindowsPath('E:/qilb研究/.qlib_data/cn_data')}


In [3]:
market = "csi300"
benchmark = "SH000300"
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}
task = {
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}
dataset = init_instance_by_config(task["dataset"])
print(type(dataset))
dtrain, dvalid, dtest = dataset.prepare(
    ["train", "valid", "test"],
    col_set=["feature", "label"],
    data_key=DataHandlerLP.DK_L
)

[15996:MainThread](2022-01-22 21:16:52,818) INFO - qlib.timer - [log.py:113] - Time cost: 66.814s | Loading data Done
[15996:MainThread](2022-01-22 21:16:53,547) INFO - qlib.timer - [log.py:113] - Time cost: 0.599s | DropnaLabel Done
E:\Anaconda3.5\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
[15996:MainThread](2022-01-22 21:16:56,115) INFO - qlib.timer - [log.py:113] - Time cost: 2.567s | CSZScoreNorm Done
[15996:MainThread](2022-01-22 21:16:56,130) INFO - qlib.timer - [log.py:113] - Time cost: 3.297s | fit & process data Done
[15996:MainThread](2022-01-22 21:16:56,130) INFO - qlib.timer - [log.py:113] - Time cost: 70.127s | Init data Done


<class 'qlib.data.dataset.DatasetH'>


In [4]:
MACD_EXP = '(EMA($close, 12) - EMA($close, 26))/$close - EMA((EMA($close, 12) - EMA($close, 26))/$close, 9)/$close'
fields = [MACD_EXP]  # MACD
names = ['MACD']
# labels = ['Ref($close, -2)/Ref($close, -1) - 1']  # label
# label_names = ['LABEL']
data_loader_config = {
    "feature": (fields, names),
    # "label": (labels, label_names)
}

In [5]:
data_loader = QlibDataLoader(config=data_loader_config)
df_train = data_loader.load(instruments='csi300', start_time='2008-01-01', end_time='2014-12-31')
df_valid = data_loader.load(instruments='csi300', start_time='2015-01-01', end_time='2016-12-31')
df_test = data_loader.load(instruments='csi300', start_time='2017-01-01', end_time='2020-08-01')

In [6]:
dtrain = dtrain.join(df_train)
dtrain = dtrain.fillna(method='bfill', axis=0).fillna(0)
dvalid = dvalid.join(df_valid)
dvalid = dvalid.fillna(method='bfill', axis=0).fillna(0)
dtest = dtest.join(df_test)
dtest = dtest.fillna(method='bfill', axis=0).fillna(0)
print(dtrain.head(5))
print([column for column in dtrain])

                        feature                                          \
                           KMID      KLEN     KMID2       KUP      KUP2   
datetime   instrument                                                     
2008-01-02 SH600000    0.010377  0.061132  0.169754  0.028302  0.462962   
           SH600004    0.057279  0.059666  0.960000  0.002387  0.040000   
           SH600006    0.012673  0.040323  0.314285  0.008065  0.200000   
           SH600007    0.066977  0.084186  0.795579  0.007907  0.093923   
           SH600008    0.051163  0.082326  0.621469  0.027907  0.338984   

                                                                         ...  \
                           KLOW     KLOW2      KSFT     KSFT2     OPEN0  ...   
datetime   instrument                                                    ...   
2008-01-02 SH600000    0.022453  0.367285  0.004528  0.074077  0.989729  ...   
           SH600004    0.000000  0.000000  0.054893  0.920001  0.945824  ...   

In [7]:
import tensorflow_cnn
model=tensorflow_cnn.CNN(epochs=5,use_own_factors=True,n_factors=159)
model.fit(dataset,own_dtrain=dtrain, own_dvalid=dvalid)

Epoch 1/5
948/948 [==============================] - 252s 265ms/step - loss: 1.2169 - val_loss: 1.0353
Epoch 2/5
948/948 [==============================] - 252s 266ms/step - loss: 1.0090 - val_loss: 1.0024
Epoch 3/5
948/948 [==============================] - 248s 262ms/step - loss: 0.9981 - val_loss: 0.9997
Epoch 4/5
948/948 [==============================] - 248s 261ms/step - loss: 0.9964 - val_loss: 1.0000
Epoch 5/5
948/948 [==============================] - 248s 262ms/step - loss: 0.9953 - val_loss: 0.9993
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 159, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 159, 16)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 159, 64)           3136      
___

In [8]:
pred=model.predict(dataset,own_test=dtest)
print(pred)

datetime    instrument
2017-01-03  SH600000         [-0.020096473]
            SH600008      [-0.000121184625]
            SH600009          [0.003464887]
            SH600010         [-0.055008233]
            SH600015         [0.0026690033]
                                ...        
2020-07-31  SZ300413        [-0.0062893843]
            SZ300433         [-0.010211444]
            SZ300498         [-0.013186752]
            SZ300601          [-0.01484087]
            SZ300628        [-0.0072812894]
Length: 252663, dtype: object
